# Raw Brain-Tumor-Progression data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ipywidgets import interact, IntSlider
from pathlib import Path

from imageprep.finder import BrainTumorProgressionDataFinder
from imageprep.pipeline import DicomImagesLoader, StackImages

## Load data

In [ ]:
dataset_path = Path("../data/raw/Brain-Tumor-Progression")
data_finder = BrainTumorProgressionDataFinder(dataset_path)

loader = DicomImagesLoader(data_finder)
loader = StackImages(loader)

loaded = list(loader.run())

key = lambda item: f"{item.labels[0]} {item.labels[1]}"
data = {key(item): {} for item in loaded}

for item in loaded:
    data[key(item)][item.labels[2]] = item.image

## Images types and shapes per study

In [ ]:
image_shapes = {
    id: {study_type: image.shape for study_type, image in studies.items()}
    for id, studies in data.items()
}

In [ ]:
image_shapes_df = pd.DataFrame(image_shapes).T.sort_index()
image_shapes_df

## Visualize studies

In [ ]:
def show_3d_image(image: np.ndarray):
    def plot_image(x: int):
        v_min = image.min()
        v_max = image.max()

        plt.imshow(image[x], vmin=v_min, vmax=v_max)
        plt.show()

    x_slider = IntSlider(min=0, max=image.shape[0] - 1, description="X")
    interact(plot_image, x=x_slider)

### 512x512 resolution examples

In [ ]:
show_3d_image(data["PGBM-004 1994-01-12"]["T1post"])

### 320x260 resolution examples

In [ ]:
show_3d_image(data["PGBM-009 1991-01-03"]["T1post"])

### 320x280 resolution examples

In [ ]:
show_3d_image(data["PGBM-003 1995-03-29"]["T1post"])

### 260x320 resolution examples

In [ ]:
show_3d_image(data["PGBM-013 1989-09-18"]["T1post"])

### 256x256 resolution examples

In [ ]:
show_3d_image(data["PGBM-011 1989-06-29"]["T1post"])

## Conclusions

Based on table and images above:
- !st dimension of the images can be 22, 23 or 24. Assume it will be of 22 (taken from the selected end),
- 2nd and 3rd dimensions of images are 512x512, 320x260, 320x280, 260x320, 256x256. It will be rescaled along 2nd dimension and cropped along 3rd dimension, to keep brain proportions,
- 15 subjects have all 10 image types, there are 16 subjects have 9 image types in common, all subjects have 6 image types in common. To consider is whether more studies or more image types is needed.